In [ ]:
#RASTREADORV5 

#NOTAÇÃO: COLOCAR PARA ANALISAR MAIS DE 5 IPS MARCAR COMO SUSPEITO. 

import pandas as pd
import tkinter as tk
from tkinter import filedialog, messagebox
from collections import defaultdict

def selecionar_arquivos():
    root = tk.Tk()
    root.withdraw()
    arquivos = filedialog.askopenfilenames(
        title="Selecione os arquivos CSV ou Excel",
        filetypes=[("Arquivos CSV ou Excel", "*.csv *.xlsx *.xls")]
    )
    return arquivos

def ler_arquivo(caminho):
    try:
        if caminho.endswith('.csv'):
            return pd.read_csv(caminho, encoding='utf-8')
        elif caminho.endswith('.xlsx') or caminho.endswith('.xls'):
            return pd.read_excel(caminho)
        else:
            return None
    except Exception as e:
        messagebox.showerror("Erro ao ler arquivo", f"{caminho}\n\n{e}")
        return None

def salvar_como_csv(df_final):
    root = tk.Tk()
    root.withdraw()
    caminho_saida = filedialog.asksaveasfilename(
        defaultextension=".csv",
        filetypes=[("CSV", "*.csv")],
        title="Salvar arquivo analisado"
    )

    if caminho_saida:
        try:
            df_final.to_csv(caminho_saida, index=False, encoding='utf-8-sig')
            messagebox.showinfo("Sucesso", f"Arquivo salvo com sucesso:\n{caminho_saida}")
        except Exception as e:
            messagebox.showerror("Erro", f"Erro ao salvar o arquivo:\n{e}")

def analisar_suspeitos(df):
    df = df.copy()
    df['FIELD'] = df['FIELD'].astype(str).str.upper()

    # Criar dicionários: {user: [ips]}, {user: [macs]}
    user_ips = defaultdict(list)
    user_macs = defaultdict(list)

    for _, row in df.iterrows():
        user = row.get("USER")
        field = row.get("FIELD")
        valor = str(row.get("ENTRADA"))

        if field == "IP":
            user_ips[user].append(valor)
        elif field == "MAC ADDRESS":
            user_macs[user].append(valor)

    # Obter IP/MAC mais frequente por usuário
    user_main_ip = {u: pd.Series(v).mode()[0] if v else None for u, v in user_ips.items()}
    user_main_mac = {u: pd.Series(v).mode()[0] if v else None for u, v in user_macs.items()}

    # Inverter: ip -> [users], mac -> [users]
    ip_to_users = defaultdict(set)
    mac_to_users = defaultdict(set)

    for u, ips in user_ips.items():
        for ip in ips:
            ip_to_users[ip].add(u)
    for u, macs in user_macs.items():
        for mac in macs:
            mac_to_users[mac].add(u)

    # Analisar cada linha
    suspeitas = []
    for _, row in df.iterrows():
        user = row.get("USER")
        field = row.get("FIELD")
        valor = str(row.get("ENTRADA"))
        motivos = []

        if user not in user_main_ip or user not in user_main_mac:
            suspeitas.append("")
            continue

        ip_main = user_main_ip[user]
        mac_main = user_main_mac[user]

        if field == "IP":
            if valor != ip_main:
                motivos.append("IP suspeito")
            if len(ip_to_users[valor]) > 1:
                motivos.append("Possível invasão")

        if field == "MAC ADDRESS":
            if valor != mac_main:
                motivos.append("MAC suspeito")
            if len(mac_to_users[valor]) > 1:
                motivos.append("Possível invasão")

        suspeitas.append(", ".join(set(motivos)))

    df['SUSPEITO'] = suspeitas
    return df

def combinar_arquivos():
    arquivos = selecionar_arquivos()
    if not arquivos:
        return

    dfs = []
    for caminho in arquivos:
        df = ler_arquivo(caminho)
        if df is not None:
            dfs.append(df)

    if not dfs:
        messagebox.showwarning("Aviso", "Nenhum arquivo válido foi carregado.")
        return

    try:
        df_final = pd.concat(dfs, ignore_index=True)
        df_final = analisar_suspeitos(df_final)
        salvar_como_csv(df_final)
    except Exception as e:
        messagebox.showerror("Erro", f"Erro ao combinar os arquivos:\n{e}")

if __name__ == "__main__":
    combinar_arquivos()


In [ ]:
#ANALISA_SUSPEITA

import pandas as pd
from tkinter import Tk, filedialog, messagebox
import csv

# Abre janela para selecionar o arquivo
Tk().withdraw()
file_path = filedialog.askopenfilename(title="Selecione o arquivo CSV", filetypes=[("CSV Files", "*.csv")])

# Tenta ler o arquivo com detecção automática de delimitador
try:
    with open(file_path, newline='', encoding='utf-8') as f:
        sample = f.read(2048)
        f.seek(0)
        dialect = csv.Sniffer().sniff(sample)
        df = pd.read_csv(f, dialect=dialect)
except Exception as e:
    try:
        df = pd.read_csv(file_path, sep=';', encoding='utf-8')
    except Exception as e_utf8:
        try:
            df = pd.read_csv(file_path, sep=';', encoding='latin1')
        except Exception as e_latin1:
            messagebox.showerror("Erro ao ler o CSV", f"Não foi possível ler o arquivo.\n\nErros:\n{e}\n{e_utf8}\n{e_latin1}")
            raise

# Cria as colunas
df['SUSPEITO'] = 'confiável'
df['MOTIVO'] = ''

# Função para analisar por usuário
def analisar_user(user_df):
    ip_entries = user_df[(user_df['FIELD'].str.contains('IP', case=False)) & (user_df['ENTRADA'].notnull())]
    mac_entries = user_df[(user_df['FIELD'].str.contains('MAC', case=False)) & (user_df['ENTRADA'].notnull())]

    ip_mais_comum = ip_entries['ENTRADA'].mode()
    mac_mais_comum = mac_entries['ENTRADA'].mode()

    ip_mais_comum = ip_mais_comum.iloc[0] if not ip_mais_comum.empty else None
    mac_mais_comum = mac_mais_comum.iloc[0] if not mac_mais_comum.empty else None

    # Regra: se mais de 4 IPs diferentes, marcar como suspeito
    ips_diferentes = ip_entries['ENTRADA'].nunique()
    if ips_diferentes >= 15:
        for idx in ip_entries.index:
            df.at[idx, 'SUSPEITO'] = 'suspeito'
            df.at[idx, 'MOTIVO'] = f"Usuário usou mais de 5 IPs diferentes (≥5)."

    for idx, row in user_df.iterrows():
        field = row['FIELD'].strip().upper()
        entrada = row['ENTRADA']

        if field.startswith('IP') and ip_mais_comum and entrada != ip_mais_comum:
            if df.at[idx, 'SUSPEITO'] != 'suspeito':
                df.at[idx, 'SUSPEITO'] = 'suspeito'
                df.at[idx, 'MOTIVO'] = f"IP diferente do mais comum ({ip_mais_comum}) para o usuário"
        elif field.startswith('MAC') and mac_mais_comum and entrada != mac_mais_comum:
            if df.at[idx, 'SUSPEITO'] != 'suspeito':
                df.at[idx, 'SUSPEITO'] = 'suspeito'
                df.at[idx, 'MOTIVO'] = f"MAC address diferente do mais comum ({mac_mais_comum}) para o usuário"

# Executa a análise por usuário
for user in df['USER'].unique():
    user_df = df[df['USER'] == user]
    analisar_user(user_df)

# Salva a nova versão
output_path = file_path.replace('.csv', '_analisado_com_motivo.csv')
df.to_csv(output_path, index=False)

print(f"Análise concluída! Arquivo salvo em: {output_path}")


: 

In [ ]:
#RELATÓRIOS

import pandas as pd
from docx import Document
import os

# Carrega o CSV com separador adequado
df = pd.read_csv("relatorio_de_credenciais.csv", sep=";", encoding="latin1")  # ou "windows-1252"

# Converte data
df["DATA"] = pd.to_datetime(df["DATA"], errors="coerce", dayfirst=True)

# Agrupa por DATA
for data_unica, grupo in df.groupby(df["DATA"].dt.date):
    doc = Document()
    doc.add_heading(f"Relatório de Acessos - {data_unica}", level=1)

    for _, linha in grupo.iterrows():
        user = linha["USER"]
        tipo = linha["TIPO"]
        field = linha["FIELD"]
        entrada = linha["ENTRADA"]
        status = linha.get("SUSPEITO", "não informado")

        motivo = " "
        if status.lower() == "suspeito":
            motivo = f"IP diferente, mas MAC ADDRESS igual - necessário revisar."
        elif status.lower() == "confiável":
            motivo = "Acesso classificado como confiável."
        elif status.lower() == "revisar":
            motivo = "IP diferente, mas MAC igual - necessário revisar."

        doc.add_paragraph(
            f"Usuário: {user}\nTipo: {tipo}\n{field}: {entrada}\nStatus: {status}\nMotivo: {motivo}\n"
        )
        doc.add_paragraph("-" * 50)

    # Cria pasta de saída
    os.makedirs("relatorios_por_data", exist_ok=True)
    nome_arquivo = f"relatorios_por_data/Relatorio_{data_unica}.docx"
    doc.save(nome_arquivo)

print("✅ Relatórios gerados com sucesso.")
